<a href="https://colab.research.google.com/github/Hniii/Big-Data/blob/main/Spark_Sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=4800006f383b41dfd5ebce02fd1893e0dec5a6dc412ec53b584e96b785dae9fb
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql.functions import *
from pyspark.sql import Row
# create a SparkSession object
spark = SparkSession.builder.appName("ReadCSV").getOrCreate()

# read CSV file into a DataFrame, assuming comma (,) as the delimiter
df = spark.read.format("csv") \
    .option("header", "false") \
    .option("inferSchema", "true") \
    .option("sep", ",") \
    .load("ngram.csv")

# Rename the columns
df = df.withColumnRenamed("_c0", "Bigram") \
        .withColumnRenamed("_c1", "Year") \
        .withColumnRenamed("_c2", "Count") \
        .withColumnRenamed("_c3", "Pages") \
        .withColumnRenamed("_c4", "Books")

# Select only the desired columns
df = df.select("Bigram", "Year", "Count", "Pages", "Books")

# show the first 20 rows of the DataFrame
df.show(40)


+--------+----+-----+-----+-----+
|  Bigram|Year|Count|Pages|Books|
+--------+----+-----+-----+-----+
|! $17.95|1985|    1|    1|    1|
|! $17.95|1987|    1|    1|    1|
|! $17.95|1990|    1|    1|    1|
|! $17.95|1991|    1|    1|    1|
|! $17.95|1992|    5|    5|    5|
|! $17.95|1993|    2|    2|    2|
|! $17.95|1995|    1|    1|    1|
|! $17.95|1996|    4|    2|    2|
|! $17.95|1997|    6|    5|    5|
|! $17.95|1998|    4|    3|    3|
|! $17.95|1999|   11|   10|   10|
|! $17.95|2000|   11|    9|    9|
|! $17.95|2001|    5|    4|    4|
|! $17.95|2002|    5|    5|    5|
|! $17.95|2003|    2|    2|    2|
|! $17.95|2004|   14|   14|   14|
|! $17.95|2005|   13|   13|   13|
|! $17.95|2006|    5|    5|    5|
|! $17.95|2007|    2|    2|    2|
|! $17.95|2008|    2|    2|    2|
|    ! 09|1780|    1|    1|    1|
|    ! 09|1803|    1|    1|    1|
|    ! 09|1806|    1|    1|    1|
|    ! 09|1823|    1|    1|    1|
|    ! 09|1824|    1|    1|    1|
|    ! 09|1825|    1|    1|    1|
|    ! 09|1829

**3.1 Retourner tous les bi-grammes dont le nombre Count est supérieur à cinq.**

In [ ]:
df.createOrReplaceTempView("ngram")

# SQL
result1 = spark.sql("Select * from ngram where Count > 5")
result1.show(40)

+--------+----+-----+-----+-----+
|  Bigram|Year|Count|Pages|Books|
+--------+----+-----+-----+-----+
|! $17.95|1997|    6|    5|    5|
|! $17.95|1999|   11|   10|   10|
|! $17.95|2000|   11|    9|    9|
|! $17.95|2004|   14|   14|   14|
|! $17.95|2005|   13|   13|   13|
|    ! 09|1899|    6|    6|    5|
|    ! 09|1916|    7|    7|    4|
|    ! 09|1936|    6|    6|    6|
|    ! 09|1997|    6|    5|    5|
|    ! 09|1999|   11|   10|   10|
|    ! 09|2000|   11|    9|    9|
|    ! 09|2004|   14|   14|   14|
|    ! 09|2005|   13|   13|   13|
+--------+----+-----+-----+-----+



In [ ]:
#  API SparkSql
df.filter("Count > 5").select("Bigram").show()

+--------+
|  Bigram|
+--------+
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|    ! 09|
|    ! 09|
|    ! 09|
|    ! 09|
|    ! 09|
|    ! 09|
|    ! 09|
|    ! 09|
+--------+



**3.2) Retourner le nombre total de bi-grammes dans chaque année.**

In [ ]:
# SQL
result2 = spark.sql("SELECT Year, COUNT(*) AS cnt FROM ngram GROUP BY Year ")
result2.show()

+----+---+
|Year|cnt|
+----+---+
|1829|  1|
|1990|  2|
|1903|  1|
|1884|  1|
|1888|  1|
|1924|  1|
|2003|  2|
|1823|  1|
|2007|  2|
|1869|  1|
|1892|  1|
|1889|  1|
|1927|  1|
|1866|  1|
|1877|  1|
|2006|  2|
|1908|  1|
|1925|  1|
|1824|  1|
|1848|  1|
+----+---+
only showing top 20 rows



In [ ]:
# SparkSQL
df.groupBy("Year").count().show()

+----+-----+
|Year|count|
+----+-----+
|1829|    1|
|1990|    2|
|1903|    1|
|1884|    1|
|1888|    1|
|1924|    1|
|2003|    2|
|1823|    1|
|2007|    2|
|1869|    1|
|1892|    1|
|1889|    1|
|1927|    1|
|1866|    1|
|1877|    1|
|2006|    2|
|1908|    1|
|1925|    1|
|1824|    1|
|1848|    1|
+----+-----+
only showing top 20 rows



**3.3) Retourner les bi-grammes qui ont le plus grand nombre de count dans chaque
année.**

In [ ]:
# SQL
result3 = spark.sql("""
  select Year,Bigram,Count from ngram as t1 where
  count = (
    select Max(Count) from ngram as t2 where
    t1.Year == t2.Year
  )
""")

# OR
result3 = spark.sql("Select Bigram, Year, Max(Count) as maxCount from ngram group by Bigram,Year")
result3.show()

+--------+----+--------+
|  Bigram|Year|maxCount|
+--------+----+--------+
|    ! 09|1848|       1|
|! $17.95|2002|       5|
|    ! 09|1881|       3|
|! $17.95|2007|       2|
|    ! 09|1910|       3|
|    ! 09|1825|       1|
|! $17.95|1985|       1|
|    ! 09|1780|       1|
|    ! 09|1876|       2|
|    ! 09|1888|       2|
|! $17.95|1990|       1|
|! $17.95|1993|       2|
|    ! 09|1878|       2|
|    ! 09|1996|       4|
|    ! 09|2007|       2|
|    ! 09|1857|       2|
|    ! 09|1911|       1|
|    ! 09|1998|       4|
|    ! 09|1908|       2|
|! $17.95|2003|       2|
+--------+----+--------+
only showing top 20 rows



In [ ]:
# SparkSQL

reslut3=df.groupBy("Year","Bigram").max("Count")
result3.show()

+--------+----+--------+
|  Bigram|Year|maxCount|
+--------+----+--------+
|    ! 09|1848|       1|
|! $17.95|2002|       5|
|    ! 09|1881|       3|
|! $17.95|2007|       2|
|    ! 09|1910|       3|
|    ! 09|1825|       1|
|! $17.95|1985|       1|
|    ! 09|1780|       1|
|    ! 09|1876|       2|
|    ! 09|1888|       2|
|! $17.95|1990|       1|
|! $17.95|1993|       2|
|    ! 09|1878|       2|
|    ! 09|1996|       4|
|    ! 09|2007|       2|
|    ! 09|1857|       2|
|    ! 09|1911|       1|
|    ! 09|1998|       4|
|    ! 09|1908|       2|
|! $17.95|2003|       2|
+--------+----+--------+
only showing top 20 rows



**3.4) Retourner tous les bi-grammes qui sont apparus dans 20 années différentes.**

In [ ]:
# SQL
result4 = spark.sql("""
  Select Bigram From ngram
  Group by Bigram
  Having Count(Distinct Year) = 20
""")
result4.show(10)

+--------+
|  Bigram|
+--------+
|! $17.95|
+--------+



In [ ]:
# SparkSQL
result = df.groupBy("Bigram").agg(
    countDistinct("Year").alias("NumYears")
).filter("NumYears = 20").select("Bigram").show()

+--------+
|  Bigram|
+--------+
|! $17.95|
+--------+



**3.5) Retourner tous les bi-grammes qui contiennent le caractère ‘!’ dans la première
partie et le caractère ‘9’ dans la deuxième partie (les deux parties sont séparées par
un espace).**

In [ ]:
# SQL
result5 = spark.sql("SELECT distinct Bigram FROM ngram WHERE Bigram LIKE '%!% %' AND Bigram LIKE '% %9%'")
result5.show(10)

+--------+
|  Bigram|
+--------+
|    ! 09|
|! $17.95|
+--------+



In [ ]:
# SparkSql
df.filter((col("Bigram").like("%!% %")) & (col("Bigram").like("% %9%"))).select("Bigram").distinct().show()

+--------+
|  Bigram|
+--------+
|    ! 09|
|! $17.95|
+--------+



**3.6) Retourner les bi-grammes qui sont apparus dans toutes les années présentes
dans les données.**

In [ ]:
# SQL
result6 = spark.sql("""
  select Bigram from ngram
  group by Bigram
  having count(distinct(Year)) = (select count(distinct(Year)) from ngram)
""")
result6.show(10)

+------+
|Bigram|
+------+
|  ! 09|
+------+



In [ ]:
# SparkSql
distinctYears =  df.select("Year").distinct().count()
df.groupBy("Bigram").agg(countDistinct('Year').alias('yearCount')) \
                    .filter(col("yearCount")==distinctYears).select("Bigram").show(10)

+------+
|Bigram|
+------+
|  ! 09|
+------+



**3.7) Retourner le nombre total de pages et de livres dans lesquels chaque bigramme apparaît pour chaque année disponible, trié par ordre alphabétique.**

In [ ]:
# Sql
result7 = spark.sql("""
  select Year, Bigram, sum(Pages) as TotalPages, sum(Books) as TotalBooks from ngram
  group by  Year, Bigram
  order by Bigram ASC
""")
result7.show(10)

+----+--------+----------+----------+
|Year|  Bigram|TotalPages|TotalBooks|
+----+--------+----------+----------+
|2002|! $17.95|         5|         5|
|1993|! $17.95|         2|         2|
|2006|! $17.95|         5|         5|
|1985|! $17.95|         1|         1|
|1996|! $17.95|         2|         2|
|1999|! $17.95|        10|        10|
|1995|! $17.95|         1|         1|
|2007|! $17.95|         2|         2|
|1991|! $17.95|         1|         1|
|2000|! $17.95|         9|         9|
+----+--------+----------+----------+
only showing top 10 rows



In [ ]:
# SparkSql
df.groupBy('Year','Bigram').agg(sum('Pages').alias('TotalPages'),sum('Books').alias('TotalBooks')) \
 .orderBy('Bigram').show()

+----+--------+----------+----------+
|Year|  Bigram|TotalPages|TotalBooks|
+----+--------+----------+----------+
|1999|! $17.95|        10|        10|
|2002|! $17.95|         5|         5|
|2006|! $17.95|         5|         5|
|1997|! $17.95|         5|         5|
|2007|! $17.95|         2|         2|
|2008|! $17.95|         2|         2|
|1985|! $17.95|         1|         1|
|1996|! $17.95|         2|         2|
|1995|! $17.95|         1|         1|
|1991|! $17.95|         1|         1|
|2000|! $17.95|         9|         9|
|1993|! $17.95|         2|         2|
|2005|! $17.95|        13|        13|
|1992|! $17.95|         5|         5|
|1987|! $17.95|         1|         1|
|1990|! $17.95|         1|         1|
|2004|! $17.95|        14|        14|
|2003|! $17.95|         2|         2|
|2001|! $17.95|         4|         4|
|1998|! $17.95|         3|         3|
+----+--------+----------+----------+
only showing top 20 rows



**3.8) Retourner le nombre total de bi-grammes différents dans chaque année, triés
par ordre décroissant de l'année.**

In [ ]:
# Sql
result8 = spark.sql("""
SELECT Year, COUNT(DISTINCT Bigram) AS TotalBigrams
FROM ngram
GROUP BY Year
ORDER BY Year DESC
""")

result8.show(10)

+----+------------+
|Year|TotalBigrams|
+----+------------+
|2008|           2|
|2007|           2|
|2006|           2|
|2005|           2|
|2004|           2|
|2003|           2|
|2002|           2|
|2001|           2|
|2000|           2|
|1999|           2|
+----+------------+
only showing top 10 rows



In [ ]:
# SparkSql
df.groupBy("Year").agg(countDistinct("Bigram").alias("TotalBigrams")).orderBy("Year", ascending=False).show()

+----+------------+
|Year|TotalBigrams|
+----+------------+
|2008|           2|
|2007|           2|
|2006|           2|
|2005|           2|
|2004|           2|
|2003|           2|
|2002|           2|
|2001|           2|
|2000|           2|
|1999|           2|
|1998|           2|
|1997|           2|
|1996|           2|
|1995|           2|
|1993|           2|
|1992|           2|
|1991|           2|
|1990|           2|
|1987|           2|
|1985|           2|
+----+------------+
only showing top 20 rows

